In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict


In [31]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

def get_dhw_profiles(xml_root):
    day_profiles = {}
    for dp in xml_root.findall(".//DHWDayProfile"):
        day_profiles[dp.attrib["id"]] = {
            "waterConsumption": float(dp.attrib["waterConsumption"]),
            "profile": [float(dp.attrib[f"p{i}"]) for i in range(1, 25)]
        }

    year_profiles = {}
    for yp in xml_root.findall(".//DHWYearProfile"):
        profile = []
        for i in range(1, 366):
            profile.append(yp.attrib.get(f"d{i}"))
        year_profiles[yp.attrib["id"]] = profile

    return day_profiles, year_profiles

def compute_building_dhw(xml_root):
    day_profiles, year_profiles = get_dhw_profiles(xml_root)
    building_profiles = {}

    for b in xml_root.findall(".//Building"):
        building_name = b.attrib["Name"]
        occupants = b.find(".//Occupants")
        
        n_occ = int(occupants.attrib["n"])
        dhw_day_id = occupants.attrib["DHWType"]
        dhw_year_id = dhw_day_id  # assuming same ID unless specified differently

        

        daily_profile = day_profiles[dhw_day_id]["profile"]
        daily_consumption = day_profiles[dhw_day_id]["waterConsumption"] * n_occ
        yearly_pattern = year_profiles[dhw_year_id]

        hourly_values = []
        for day_id in yearly_pattern:
            
            p = day_profiles[day_id]["profile"]
            c = day_profiles[day_id]["waterConsumption"] * n_occ
            hourly_values.extend([c * val for val in p])

        series = pd.Series(hourly_values, name=building_name)
        building_profiles[building_name] = series

    return building_profiles
def convert_water_to_energy_kwh(series, target_temp=55, inlet_temp=10, c=4180):
    delta_T = target_temp - inlet_temp  # °C
    energy_kwh = series * c * delta_T / 3_600_000  # J to kWh
    return energy_kwh

# Load and parse the XML
path_to_file = r'C:\Users\alejandr.penabell\Downloads\simulations (1)\simulations\val_de_bagnes_41_climate_contemporary_pv_roof_2025\simulation.xml'
tree = ET.parse(path_to_file)
root = tree.getroot()

# Compute DHW profiles
building_dhw_profiles = compute_building_dhw(root)
building_dhw_energy_profiles_kwh = {
    name: convert_water_to_energy_kwh(profile)
    for name, profile in building_dhw_profiles.items()
}

# Display a sample building profile as requested
sample_profile = next(iter(building_dhw_energy_profiles_kwh.values()))
print(sample_profile.sum())

# 953 kWh for 50 l per person per year at 55°C



4767.812500000001
